### Проект: "RFM сегментация пользователей"

### Цель проекта: отбор наиболее лояльных пользователей для акции в следующем месяце

В данном проекте будет использован датасет маркетплейса по покупкам пользователей за период с 2021-01	по 2022-04.

Описание данных:

* transaction_id - айди транзакции;
* created_at - время создания заказа;
* user_id - айди пользователя;
* amount - сумма заказа, руб.

__Подключим необходимые библиотеки для дальнейшей работы__

In [1]:
import pandas as pd
import sqlite3
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls /content/drive/MyDrive/datasets/marketplace_users/

transactions.csv  users.csv


In [3]:
trans = pd.read_csv('drive/MyDrive/datasets/marketplace_users/transactions.csv')

__Считаем данные и убедимся, что все корректно__

In [4]:
trans.head()

,Unnamed: 0,transaction_id,created_at,user_id,amount
0,1,e8152ef72822baae14866c30d5155ec8,2022-04-27 21:58:28,e1d22453e5f60161,213
1,2,9305700c4b7364cda47edaf6990ab7bc,2022-02-19 14:57:37,e1d22453e5f60161,128
2,3,3e89681d4f3d62beed4db199c719da8f,2021-12-09 20:40:10,e1d22453e5f60161,391
3,4,50c2d79c35513eaa1864999106f91dc3,2022-04-30 09:09:54,e08aa20152ca92c7,163
4,5,ce34444f31415fe2993d5b1dbe2ac846,2021-11-05 05:45:31,e08aa20152ca92c7,329


In [5]:
trans.shape # размерность датасета

(75059, 5)

In [6]:
trans.isna().sum() # проверка на пропуски в данных

Unnamed: 0        0
transaction_id    0
created_at        0
user_id           0
amount            0
dtype: int64

In [7]:
trans.dtypes

Unnamed: 0         int64
transaction_id    object
created_at        object
user_id           object
amount             int64
dtype: object

__Распарсим дату__

In [8]:
trans['created_at'] = pd.to_datetime(trans['created_at'],format='%Y/%m/%d %H:%M:%S')

In [9]:
trans.dtypes

Unnamed: 0                 int64
transaction_id            object
created_at        datetime64[ns]
user_id                   object
amount                     int64
dtype: object

__Создадим подключение к SQLite3 для удобной трансформации данных__

In [10]:
conn = sqlite3.connect('db')
cur = conn.cursor()

In [11]:
def select(sql):
  return pd.read_sql(sql, conn)

In [ ]:
trans.to_sql('transactions', conn, index=False, if_exists='replace')

__Взглянем на количество заказов по месяцам:__

In [13]:
sql = ''' select

date(t.created_at, 'start of month') as month,
count(*) as purch_count_per_month

from transactions t
group by date(t.created_at, 'start of month')
order by month
'''

In [14]:
select(sql) # используем данные с даты 2021-11-01, другими словами, будем опираться на полугодовые данные по пользователям

,month,purch_count_per_month
0,2021-01-01,217
1,2021-02-01,635
2,2021-03-01,1050
3,2021-04-01,1497
4,2021-05-01,2154
5,2021-06-01,2658
6,2021-07-01,3313
7,2021-08-01,4041
8,2021-09-01,4713
9,2021-10-01,5684


In [15]:
sql = ''' select

max(t.created_at) as last_purchase

from transactions t
'''

In [16]:
select(sql)

,last_purchase
0,2022-04-30 23:59:45


__Последняя покупка была 2022-04-30, следовательно, дату "сегодня" для дальнейшего анализа установим как 2022-05-01__

In [17]:
sql = '''

select 

t.user_id,
max(t.created_at) as last_purchase,
date('2022-05-01') as today,
julianday(date('2022-05-01')) - julianday(max(t.created_at)) as recency,
count(distinct t.transaction_id) as frequency,
sum(t.amount) as monetary

from transactions t

where t.created_at > "2021-11-01 00:00:01" 
group by t.user_id
'''

In [18]:
t = select(sql)

__Получили таблицу с дополнительными полями:__
* _last_purchase_ (последняя покупка клиента);
* _today_ (дата "сегодня");
* _recency_ (как давно была последняя покупка); 
* _frequency_ (общее число покупок за полгода); 
* _monetary_ (общая сумма потраченных денег за полгода, руб) 

In [19]:
t.head()

,user_id,last_purchase,today,recency,frequency,monetary
0,00027f46f0e4e4c4,2022-04-22 20:12:39,2022-05-01,8.157882,9,28289
1,00054380944abc67,2022-03-11 02:48:43,2022-05-01,50.882836,5,10709
2,0014d31816486e07,2022-04-07 17:06:59,2022-05-01,23.286817,9,3599
3,00162b63a0b32f04,2022-03-16 09:20:28,2022-05-01,45.610787,4,3017
4,0021a1b1ae8e71cb,2021-11-22 00:34:04,2022-05-01,159.976343,1,3463


In [20]:
t['r'] = 5 - (pd.qcut(t['recency'], 4, labels=False) + 1)

In [21]:
t['f'] = pd.qcut(t['frequency'], 4, labels=False, duplicates='drop') + 1

In [22]:
t['f'].value_counts()

1    2357
3    1614
4    1466
2    1113
Name: f, dtype: int64

In [23]:
def freq_dist(f_count):
  if f_count in [1, 2, 3, 4, 5]:
    return 1
  if f_count in [6, 7, 8, 9, 10]:
    return 2
  if f_count in [11, 12, 13, 14, 15, 16, 17, 18]:
    return 3
  else:
    return 4

In [24]:
t['f'] = t['frequency'].apply(freq_dist)

In [25]:
t.groupby('f')['frequency'].agg(['mean','count'])

,mean,count
f,,
1,2.787320,3470
2,7.624535,1614
3,13.874608,957
4,27.192534,509


In [26]:
t['m'] = pd.qcut(t['monetary'], 4, labels=False) + 1

In [27]:
# нормированный скор клиента
t['rfm_score'] = ((t['r'] + t['f'] + t['m']) / 3).round(2)

In [28]:
# "классический" скор клиента
t['rfm_score_conc'] = t['r'].astype(str) + t['f'].astype(str) + t['m'].astype(str)

In [29]:
t.head()

,user_id,last_purchase,today,recency,frequency,monetary,r,f,m,rfm_score,rfm_score_conc
0,00027f46f0e4e4c4,2022-04-22 20:12:39,2022-05-01,8.157882,9,28289,3,2,4,3.00,324
1,00054380944abc67,2022-03-11 02:48:43,2022-05-01,50.882836,5,10709,1,1,3,1.67,113
2,0014d31816486e07,2022-04-07 17:06:59,2022-05-01,23.286817,9,3599,2,2,2,2.00,222
3,00162b63a0b32f04,2022-03-16 09:20:28,2022-05-01,45.610787,4,3017,2,1,2,1.67,212
4,0021a1b1ae8e71cb,2021-11-22 00:34:04,2022-05-01,159.976343,1,3463,1,1,2,1.33,112


__Отберем наиболее лояльных клиентов по получившемуся анализу__

In [30]:
best_users = t.query('rfm_score == 4.0') # or column 'rfm_score_conc == 444'

In [31]:
best_users.shape

(300, 11)

In [32]:
best_users[['recency', 'frequency', 'monetary']].mean().round(2)

recency          3.06
frequency       29.66
monetary     36128.07
dtype: float64

In [33]:
print(f"Таким образом, получили таблицу с", best_users.shape[0], "id клиентов с самым высоким rfm_score, которые:")
print(f"больше тратят (в среднем", best_users[['recency', 'frequency', 'monetary']].mean().round(2)[2], "руб. за период),")
print(f"чаще покупают (в среднем", best_users[['recency', 'frequency', 'monetary']].mean().round(2)[1], "раз за период/каждые 6.21 дня),")
print(f"последняя покупка была недавно (в среднем", best_users[['recency', 'frequency', 'monetary']].mean().round(2)[0], "дня назад).")

Таким образом, получили таблицу с 300 id клиентов с самым высоким rfm_score, которые:
больше тратят (в среднем 36128.07 руб. за период),
чаще покупают (в среднем 29.66 раз за период/каждые 6.21 дня),
последняя покупка была недавно (в среднем 3.06 дня назад).
